# Example 004 - load and process data from Nikon N-SIM instrument
  
Load raw Nikon N-SIM 2D-SIM data, determine SIM pattern infromation, and run reconstruction.

Samples prepared by and imaging data acquired by Dr. Christophe Leterrier. COS cells immunofluorescence labeled for microtubules (640 nm ex.), clathrin (561 nm ex.), and actin (488 nm ex.).


### Import libraries

In [ ]:
import numpy as np
from numpy import fft
from pathlib import Path
import mcsim.analysis.sim_reconstruction as sim
from mcsim.analysis import analysis_tools
import matplotlib.pyplot as plt
from matplotlib.colors import PowerNorm
import napari
import nd2

### Define data path

In [ ]:
nd2_path = Path("data", "example_004", "raw_data", "neurocyte_lab_NSIM_data.nd2")

### Parse raw Nikon N-SIM data

In [ ]:
# Nikon 2D SIM phases and angles
n_phases = 3
n_angles = 3

# open ND2File
try:
    f = nd2.ND2File(nd2_path)
except:
    raise Exception('Cannot open specified file.')
    
# extract total image size
# in N-SIM data, all 9 images are in one giant tiled image.
# rows are angles, columns are phases
height_pixels = f.attributes.heightPx
width_pixels = f.attributes.widthPx

# determine individual image size
n_ypixels = int(height_pixels/n_angles)
n_xpixels = int(height_pixels/n_phases)

# extract experiment metadata
sizes = f.sizes
try:
    n_channels = int(sizes['C'])
except:
    n_channels = 1
try:
    n_z_steps = int(sizes['Z'])
except:
    n_z_steps = 1  

# extract per channel metadata
wvl_ch_um = np.zeros(n_channels)
NA_ch = np.zeros(n_channels)
RI_ch = np.zeros(n_channels)
pixel_size_ch_um = np.zeros([n_channels,3])
for i in range(n_channels):
    wvl_ch_um[i] = f.metadata.channels[i].channel.emissionLambdaNm / 1e3
    NA_ch[i] = f.metadata.channels[i].microscope.objectiveNumericalAperture
    RI_ch[i] = f.metadata.channels[i].microscope.immersionRefractiveIndex
    pixel_size_ch_um[i,:] = np.flip(f.metadata.channels[i].volume.axesCalibration)

# load data into memory
data = f.asarray()

# close ND2File
f.close()

# cut up raw ND2 data into individual channels, z planes, angles, and phases
sim_images = np.zeros((n_channels,n_angles,n_phases,n_z_steps,n_ypixels,n_xpixels),dtype=np.uint16)
for channel_idx in range(n_channels):
    if n_z_steps == 1:
        for angle_idx in range(n_angles):
            for phase_idx in range(n_phases):
                x_min = 0+(phase_idx*n_xpixels)
                x_max = x_min + n_xpixels
                y_min = 0+(angle_idx*n_ypixels)
                y_max = y_min + n_ypixels
                sim_images[channel_idx,angle_idx,phase_idx,0,:,:]=data[channel_idx,y_min:y_max,x_min:x_max]
    else:
        for z_idx in range(n_z_steps):
            for angle_idx in range(n_angles):
                for phase_idx in range(n_phases):
                    x_min = 0+(phase_idx*n_xpixels)
                    x_max = x_min + n_xpixels
                    y_min = 0+(angle_idx*n_ypixels)
                    y_max = y_min + n_ypixels
                    sim_images[channel_idx,angle_idx,phase_idx,z_idx,:,:]=data[channel_idx,z_idx,y_min:y_max,x_min:x_max]

### Visualize raw Nikon N-SIM data

In [ ]:
# add images to napari viewer with scale information
viewer = napari.view_image(data,name='Raw N-SIM',scale=(1,pixel_size_ch_um[0,1],pixel_size_ch_um[0,2]))
viewer.add_image(sim_images,name='Parsed raw N-SIM',scale=(1,1,1,pixel_size_ch_um[0,1],pixel_size_ch_um[0,2]))

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

# label sliders in napari viewer
viewer.dims.axis_labels = ['c_parsed','a','p','c_raw','y','x']

In [ ]:
plt.imshow(sim_images[0, 0, 0, 0])
plt.show()

### Calculate FFT of raw data to find frequency guesses

In [ ]:
%matplotlib widget
z_idx=0
for ch_idx in range(n_channels):

    image_set = sim_images[ch_idx,:,:,z_idx,:,:]

    dx = np.round(pixel_size_ch_um[ch_idx,1],3)
    nx = n_xpixels
    ny = n_ypixels
    fxs = analysis_tools.get_fft_frqs(nx, dx)
    df = fxs[1] - fxs[0]
    fys = analysis_tools.get_fft_frqs(ny, dx)
    ff = np.sqrt(np.expand_dims(fxs, axis=0)**2 + np.expand_dims(fys, axis=1)**2)
    for ii in range(image_set.shape[0]):
        ft = fft.fftshift(fft.fft2(fft.ifftshift(np.squeeze(image_set[ii, 0, :]))))

        figh = plt.figure()
        plt.title('Channel='+str(ch_idx)+", Angle="+str(ii))
        plt.imshow(np.abs(ft), norm=PowerNorm(gamma=0.1,vmin=65,vmax=1.5e7),
                    extent=[fxs[0] - 0.5 * df, fxs[-1] + 0.5 * df, fys[-1] + 0.5 * df, fys[0] - 0.5 * df])
        plt.show()

### Define correct guesses for frequencies and phases based on FFT

In [ ]:
# frequency guesses from FFTs
# 2D-SIM pattern angles are commonly 60 degrees (pi/3) rotated. Frequency, order, and relative rotation angle are often unknown.
frequencies_ch = []
frequencies_ch.append([[-3.3,-0.92], [-2.4,2.4], [-.85,-3.3]]) # ch 0
frequencies_ch.append([[-3.7,-1.1], [-2.76,2.7], [-.95,-3.7]]) # ch 1
frequencies_ch.append([[-4.6,-1.3], [-3.45,3.3], [-1.2,-4.6]]) # ch 2

# phases guesses
# 2D-SIM pattern phases are 120 degrees (2*pi/3) apart. Order and relative phase shift (for each angle) are often unknown.
phases_ch =[]
phases_ch.append([[0, 4*np.pi/3, 2*np.pi/3],[0, 4*np.pi/3, 2*np.pi/3],[0, 4*np.pi/3, 2*np.pi/3]]) # ch 0
phases_ch.append([[0, 4*np.pi/3, 2*np.pi/3],[0, 4*np.pi/3, 2*np.pi/3],[0, 4*np.pi/3, 2*np.pi/3]]) # ch 1
phases_ch.append([[0, 4*np.pi/3, 2*np.pi/3],[0, 4*np.pi/3, 2*np.pi/3],[0, 4*np.pi/3, 2*np.pi/3]]) # ch 2

### Define SIM reconstruction parameters for all channels

In [ ]:
# define parameters for channels in order: [ch0,ch1,ch2]
wiener_parameter=[0.4,0.4,.4]
band0_exclusion_fraction=[0.5,0.5,0.5]
backgrounds = [100,100,100]

### Demonstrate SIM reconstruction output with incorrect parameters

In [ ]:
# frequency guesses from FFTs -> use incorrect guess to demonstrate diagnostic output
frequencies_ch_incorrect_guess = []
frequencies_ch_incorrect_guess.append([[-5,5], [-2,-4], [-3.5,4.5]]) # ch 0 intentionally incorrect guess
frequencies_ch_incorrect_guess.append([[-3.7,-1.1], [-2.76,2.7], [-.95,-3.7]]) # ch 1
frequencies_ch_incorrect_guess.append([[-4.6,-1.3], [-3.45,3.3], [-1.2,-4.6]]) # ch 2

# phases guesses -> use incorrect guess to demonstrate diagnostic output
phases_ch_incorrect_guess =[]
phases_ch_incorrect_guess.append([[2*np.pi/3, 0, 4*np.pi/3],[2*np.pi/3, 0, 4*np.pi/3],[2*np.pi/3, 0, 4*np.pi/3]]) # ch 0 intentionally incorrect guess
phases_ch_incorrect_guess.append([[0, 4*np.pi/3, 2*np.pi/3],[0, 4*np.pi/3, 2*np.pi/3],[0, 4*np.pi/3, 2*np.pi/3]]) # ch 1
phases_ch_incorrect_guess.append([[0, 4*np.pi/3, 2*np.pi/3],[0, 4*np.pi/3, 2*np.pi/3],[0, 4*np.pi/3, 2*np.pi/3]]) # ch 2

%matplotlib inline
# path to save data
save_path = Path("data","example_004","reconstruction_incorrect_guess")

# loop over all channels and z planes
n_channels_limited = 1
for ch_idx in range(n_channels_limited):
    for z_idx in range(n_z_steps):

        # define dictionary with physical parameters of system
        physical_params = {'pixel_size': np.round(pixel_size_ch_um[ch_idx,1],3),
                                        'na': NA_ch[ch_idx],
                                        'wavelength': wvl_ch_um[ch_idx]}

        # extract all angles/phases for current channel and z plane
        sim_individual_channel = sim_images[ch_idx,:,:,z_idx,:,:]

        # create mcSIM reconstruction object. See docstring for details on each parameter.
        imgset = sim.SimImageSet(physical_params,
                                sim_individual_channel,
                                frq_guess=frequencies_ch_incorrect_guess[ch_idx],
                                phases_guess=phases_ch_incorrect_guess[ch_idx],
                                determine_amplitudes=True,
                                wiener_parameter=wiener_parameter[ch_idx],
                                fmax_exclude_band0=band0_exclusion_fraction[ch_idx],
                                phase_estimation_mode="real-space", # use real-space for Nikon data, wicker-iterative fails.
                                gain=1,
                                background=backgrounds[ch_idx],
                                min_p2nr=0.8,
                                max_phase_err=30*np.pi/180, # increase maximum phase error due to lack of calibration
                                save_dir=save_path,
                                save_suffix='_z'+str(z_idx).zfill(3)+'_ch'+str(ch_idx).zfill(3))

            
        # perform reconstruction, plot figures, save, and clean up log file
        imgset.reconstruct()
        imgset.plot_figs()
        imgset.save_imgs()
        imgset.save_result()

        # create variables to hold widefield and SIM SR images
        if ch_idx == 0 and z_idx == 0:
            wf_images_incorrect_guess = np.zeros((n_channels,n_z_steps,imgset.widefield.shape[0],imgset.widefield.shape[1]),dtype=np.float32)
            SR_images_incorrect_guess = np.zeros((n_channels,n_z_steps,imgset.sim_sr.shape[0],imgset.sim_sr.shape[1]),dtype=np.float32)

        # store widefield and SIM SR images for display
        wf_images_incorrect_guess[ch_idx,z_idx,:]=imgset.widefield
        SR_images_incorrect_guess[ch_idx,z_idx,:]=imgset.sim_sr

        # clean up mcSIM reconstruction object
        del sim_individual_channel, imgset

### Perform SIM reconstruction with correct parameters

In [ ]:
%matplotlib inline
# path to save data
save_path = Path("data","example_004","reconstruction")

# loop over all channels and z planes
for ch_idx in range(n_channels):
    for z_idx in range(n_z_steps):

        # define dictionary with physical parameters of system
        physical_params = {'pixel_size': np.round(pixel_size_ch_um[ch_idx,1],3),
                                        'na': NA_ch[ch_idx],
                                        'wavelength': wvl_ch_um[ch_idx]}

        # extract all angles/phases for current channel and z plane
        sim_individual_channel = sim_images[ch_idx,:,:,z_idx,:,:]

        # create mcSIM reconstruction object. See docstring for details on each parameter.
        imgset = sim.SimImageSet(physical_params,
                                sim_individual_channel,
                                frq_guess=frequencies_ch[ch_idx],
                                phases_guess=phases_ch[ch_idx],
                                determine_amplitudes=True,
                                wiener_parameter=wiener_parameter[ch_idx],
                                fmax_exclude_band0=band0_exclusion_fraction[ch_idx],
                                phase_estimation_mode="real-space", # use real-space for Nikon data, wicker-iterative fails.
                                gain=1,
                                background=backgrounds[ch_idx],
                                min_p2nr=0.5,
                                max_phase_err=30*np.pi/180, # increase maximum phase error due to lack of calibration
                                save_dir=save_path,
                                save_suffix='_z'+str(z_idx).zfill(3)+'_ch'+str(ch_idx).zfill(3))

            
        # perform reconstruction, plot figures, save, and clean up log file
        imgset.reconstruct()
        imgset.plot_figs()
        imgset.save_imgs()
        imgset.save_result()

        # create variables to hold widefield and SIM SR images
        if ch_idx == 0 and z_idx == 0:
            wf_images = np.zeros((n_channels,n_z_steps,imgset.widefield.shape[0],imgset.widefield.shape[1]),dtype=np.float32)
            SR_images = np.zeros((n_channels,n_z_steps,imgset.sim_sr.shape[0],imgset.sim_sr.shape[1]),dtype=np.float32)

        # store widefield and SIM SR images for display
        wf_images[ch_idx,z_idx,:]=imgset.widefield
        SR_images[ch_idx,z_idx,:]=imgset.sim_sr

        # clean up mcSIM reconstruction object
        del sim_individual_channel, imgset

### Display results

In [ ]:
# define colormaps
colormaps = ['bop purple', 'bop blue', 'bop orange']

# add images to napari viewer with scale information, colormaps, and additive blending
for ch_idx in range(n_channels):
    if ch_idx == 0:
        viewer = napari.view_image(wf_images[ch_idx,:],name='Widefield CH '+str(ch_idx).zfill(2),scale=(pixel_size_ch_um[ch_idx,1],pixel_size_ch_um[ch_idx,2]),colormap = colormaps[ch_idx],blending='additive',contrast_limits=[0,2**16-1])
        viewer.add_image(SR_images[ch_idx,:],name='SR CH '+str(ch_idx).zfill(2),scale=(pixel_size_ch_um[ch_idx,1]/2,pixel_size_ch_um[ch_idx,2]/2),colormap = colormaps[ch_idx],blending='additive',contrast_limits=[0,2**16-1])
    else:
        viewer.add_image(wf_images[ch_idx,:],name='Widefield CH '+str(ch_idx).zfill(2),scale=(pixel_size_ch_um[ch_idx,1],pixel_size_ch_um[ch_idx,2]),colormap = colormaps[ch_idx],blending='additive',contrast_limits=[0,2**16-1])
        viewer.add_image(SR_images[ch_idx,:],name='SR CH '+str(ch_idx).zfill(2),scale=(pixel_size_ch_um[ch_idx,1]/2,pixel_size_ch_um[ch_idx,2]/2),colormap = colormaps[ch_idx],blending='additive',contrast_limits=[0,2**16-1])

# activate scale bar in physical units
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True